In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

from tqdm import tqdm 
from openpyxl import load_workbook, Workbook
import configparser
from urllib import parse
import json
import random
from urllib.parse import unquote
import os
import time
from bs4 import BeautifulSoup
import requests
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS += 'HIGH:!DH:!aNULL'
try:
    requests.packages.urllib3.contrib.pyopenssl.DEFAULT_SSL_CIPHER_LIST += 'HIGH:!DH:!aNULL'
except AttributeError:
    # no pyopenssl support used / needed / available
    pass

def get_driver():

    options = webdriver.ChromeOptions()
    options.add_argument("disable-gpu")
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    
    #driver = webdriver.Chrome()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    return driver


def save_excel(_FILENAME, _DATA, _HEADER):
    if _HEADER == None:
        print(">>> 헤더 리스트를 먼저 넣어주세요")
        return None
    book = Workbook()
    sheet = book.active
    sheet.title = 'result'
    sheet.append(_HEADER)
    for d in _DATA:
        sheet.append(d)
    #sheet.column_dimensions['A'].width = 40
    book.save(_FILENAME)

# 환경설정부분(AREA 부분만수정)

In [11]:
#===== CONFIG AREA START
KEYWORD = 'virtual influencer'
MAX_PAGE = 5
SAVE_FILE = '테스트230909.xlsx'
#===== CONFIG AREA END
driver = get_driver()
driver.get('https://www.google.com/search?newwindow=1&sca_esv=562316449&hl=ko&sxsrf=AB5stBh2NdA1gK4MOl0NyORpX-whxoGxjA:1693734336307&q=.&tbm=nws&source=lnms&sa=X&ved=2ahUKEwir_dWLlI6BAxV3s1YBHTB-Cl8Q0pQJegQICRAB&biw=1452&bih=917&dpr=1')

# 코드실행부분

In [12]:
"""검색하는 부분 """
search_el = driver.find_element(By.XPATH, '//*[@id="APjFqb"]')
search_el.clear()
search_el.send_keys(KEYWORD+'\n')
time.sleep(3)

"""수집부분"""
print(">>> 검색 시작")
link_list = []
# 첫페이지 수집 후 
bs = BeautifulSoup(driver.page_source, 'lxml')
div = bs.find('div', id='rso')
for a in div.find_all('a'):
    link_list.append(a['href'])

# 2페이지부터 수집 
for page_idx in tqdm(range(MAX_PAGE)):
    if page_idx == 0:
        continue # 첫페이지는 파밍완료
    # 넘어가기 
    try:
        next_el = driver.find_element(By.XPATH, '//*[@id="pnnext"]/span[2]')
    except: # 다음버튼을 못 누르는 경우 
        break
    next_el.click()
    time.sleep(2)
    
    # 현재페이지 수집
    bs = BeautifulSoup(driver.page_source, 'lxml')
    div = bs.find('div', id='rso')
    for a in div.find_all('a'):
        link_list.append(a['href'])
        
"""수집한 링크들 방문하여 기사본문들 수거"""
print(">>> 기사 수집 시작")
본문리스트 = [] 
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
}
for link in tqdm(link_list):
    resp = requests.get(link, headers=headers)
    bs = BeautifulSoup(resp.text, 'lxml')
    본문 = bs.body.text.strip().replace('\n\n',' ')
    본문리스트.append(본문)
    
"""저장"""
print(">>> 저장 시작")
os.makedirs(f"./datas/{KEYWORD}", exist_ok=True)
data_list = [] 

for idx in range(len(link_list)):
    with open(f'./datas/{KEYWORD}/{idx}.txt', 'w', encoding='utf-8') as f:
        f.write(본문리스트[idx])
    data_list.append([f"{idx}.txt", KEYWORD, link_list[idx]])
save_excel(f"./datas/{KEYWORD}/{SAVE_FILE}", data_list, ['파일명', '키워드', 'URL'])
print(f"./datas/{KEYWORD}/{SAVE_FILE} 저장완료")

>>> 검색 시작


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:12<00:00,  2.46s/it]


>>> 기사 수집 시작


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:54<00:00,  1.09s/it]

>>> 저장 시작
./datas/virtual influencer/테스트230909.xlsx 저장완료
